In [ ]:
"""
from pyspark import SparkConf, SparkContext
conf = SparkConf().setAppName("Barcelona_Bike_Sharing")
sc = SparkContext(conf = conf)
"""

In [1]:
print(sc.version)

4.0.0


In [8]:
inputStations = "Data/stations.csv"
inputRegister = "Data/registerSample.csv"
outputPath = "Result/"